I'm importing all the required libraries and setting the parameters for the graph

In [ ]:
import numpy as np
import sep
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.patches import Ellipse

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

I am opening the image and getting the data from the image.

In [ ]:
# open the file
hdul = fits.open("image.fits")
# get the data
data = hdul[0].data

I am creating a graph using the mean and standard deviation from the data. 

In [ ]:
# shows the image
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar()
plt.savefig("Tutorial_1.png",bbox_inches='tight',dpi=400)

First we are getting the background of the image, then we are printing the mean and noise of the background. After that, we are making a 2d array that has the same size as the original image for the background. This is for back, which is the global mean. Finally, we are creating a graph to show the array. 

In [ ]:
# measure a spatially varying background on the image
bkg = sep.Background(data)

# get a "global" mean and noise of the image background:
print(bkg.globalback)
print(bkg.globalrms)

# evaluate background as 2-d array, same size as original image
bkg_image = bkg.back()
# bkg_image = np.array(bkg) # equivalent to above

# show the background
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()
plt.savefig("Tutorial_2.png",bbox_inches='tight',dpi=400)

We are making a 2d array that has the same size as the original image for the background. This is for rms, which is the noise of the image background. We are also creating a graph to show the array. 

In [ ]:

# evaluate the background noise as 2-d array, same size as original image
bkg_rms = bkg.rms()

# show the background noise
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()
plt.savefig("Tutorial_3.png",bbox_inches='tight',dpi=400)

We are removing the background of the image from data. We are also extracting and checking how many objects are in the image. 

In [ ]:
# subtract the background
data_sub = data - bkg

objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

# how many objects were detected
len(objects)

We are creating a plot that circles all the objects that are detected.

In [ ]:
# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
plt.savefig("Tutorial_4.png",bbox_inches='tight',dpi=400)

We are performing Aperture Photometry.

In [ ]:
# available fields
objects.dtype.names
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

printing the flux of the first 10 objects


In [ ]:
# show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

hdul.close()